# Backpropagation Practice

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [6]:
# training data

import numpy as np

X = np.array([[0,0,1],[0,1,1],[1,0,1],[0,1,0], [1,0,0], [1,1,1], [0,0,0]])
y = [[0], [1], [1], [1], [1], [0], [0]]


In [3]:
class NeuralNetwork:
    def __init__(self):
        # Set up Architecture of Neural Network
        self.input = 3
        self.hiddenNodes = 4
        self.outputNodes = 1
        
        # Initial Weights
        # 3x4 Matrix Array for the First Layer
        self.weights1 = np.random.randn(self.input,self.hiddenNodes)
        # 4x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)

In [4]:
nn = NeuralNetwork()

print("Layer 1 weights: \n", nn.weights1)
print("Layer 2 weights: \n", nn.weights2)

Layer 1 weights: 
 [[-0.09576852 -0.79719475 -0.08461359 -0.89554281]
 [-0.50106216 -0.46304678  0.85022985 -0.67078977]
 [-0.88560674 -0.30941571 -0.68014494  0.42957375]]
Layer 2 weights: 
 [[-1.41734931]
 [ 0.88558848]
 [-0.34277147]
 [ 0.41367125]]


In [8]:
class NeuralNetwork:
    def __init__(self):
        # Set up Architecture of Neural Network
        self.input = 3
        self.hiddenNodes = 4
        self.outputNodes = 1
        
        # Initial Weights
        # 3x4 Matrix Array for the First Layer
        self.weights1 = np.random.randn(self.input,self.hiddenNodes)
        # 4x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
    
    def sigmoid(self, x):
        return 1 / (1+np.exp(-x))
    
    def feed_forward(self,X):
        """
        Calculate the NN inference using feed forward.
        """
        
        # Weighted sum of inputs & hidden
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final Activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output

In [9]:
# generate an output

nn = NeuralNetwork()

print(X[0])
output = nn.feed_forward(X[0])
print("output", output)

[0 0 1]
output [0.28078773]


In [11]:
# calculate error

output_all = nn.feed_forward(X)
error_all = y - output_all
print(output_all)
print(error_all)

[[0.28078773]
 [0.36811065]
 [0.2645386 ]
 [0.35927458]
 [0.26308314]
 [0.36424986]
 [0.28351458]]
[[-0.28078773]
 [ 0.63188935]
 [ 0.7354614 ]
 [ 0.64072542]
 [ 0.73691686]
 [-0.36424986]
 [-0.28351458]]


In [12]:
class NeuralNetwork:
    def __init__(self):
        # Set up Architecture of Neural Network
        self.input = 3
        self.hiddenNodes = 4
        self.outputNodes = 1
        
        # Initial Weights
        # 3x4 Matrix Array for the First Layer
        self.weights1 = np.random.randn(self.input,self.hiddenNodes)
        # 4x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
    
    def sigmoid(self, x):
        return 1 / (1+np.exp(-x))
    
    def sigmoidPrime(self, x):
        return x * (1 - x)
    
    def feed_forward(self,X):
        """
        Calculate the NN inference using feed forward.
        """
        
        # Weighted sum of inputs & hidden
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final Activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backward(self, X, y, o):
        """
        Backward propagate through the network
        """
        self.o_error = y - o #error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # apply derivative of sigmoid to error
        
        self.z2_error = self.o_delta.dot(self.weights2.T) # z2 error: how much our hidden layer weights were off
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden)
        
        self.weights1 += X.T.dot(self.z2_delta) #Adjust first set (input => hidden) weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta) #adjust second set (hidden => output) weights
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [14]:
nn = NeuralNetwork()

for i in range(1000):
    if ((i+1) % 100 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X,y)

+---------EPOCH 100---------+
Predicted Output: 
 [[0.13544093]
 [0.67572397]
 [0.65640884]
 [0.8272057 ]
 [0.80602869]
 [0.64864557]
 [0.2346043 ]]
Loss: 
 0.11211672304612329
+---------EPOCH 200---------+
Predicted Output: 
 [[0.05819767]
 [0.7996175 ]
 [0.79737584]
 [0.86177908]
 [0.86032445]
 [0.32554651]
 [0.1022775 ]]
Loss: 
 0.03423602379754021
+---------EPOCH 300---------+
Predicted Output: 
 [[0.03349886]
 [0.87698323]
 [0.87649251]
 [0.90677842]
 [0.90659006]
 [0.18983864]
 [0.07002269]]
Loss: 
 0.012838137822597804
+---------EPOCH 400---------+
Predicted Output: 
 [[0.02395483]
 [0.90737594]
 [0.90712577]
 [0.92747625]
 [0.92740037]
 [0.14080384]
 [0.05688848]]
Loss: 
 0.007338727684544891
+---------EPOCH 500---------+
Predicted Output: 
 [[0.01900635]
 [0.92357648]
 [0.92340883]
 [0.93907398]
 [0.93903086]
 [0.11532392]
 [0.04931362]]
Loss: 
 0.005032665493381891
+---------EPOCH 600---------+
Predicted Output: 
 [[0.0159613 ]
 [0.93382366]
 [0.93369661]
 [0.94660803]
 [0.94

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [ ]:
##### Your Code Here #####

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?